In [25]:
import pandas
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [28]:
df_BTC = pd.read_csv('/Users/jack/Desktop/Quant Trade Comp/Data/1hr/BTCUSDT_1h.csv')
df_DOGE = pd.read_csv('/Users/jack/Desktop/Quant Trade Comp/Data/1hr/DOGEUSDT_1h.csv')

# Trend Detection

### Calculate SMA

In [32]:
def calculate_SMA20(df, column_name='close', period = 20):

    return df[column_name].rolling(window=period).mean()

def calculate_SMA50(df, column_name='close', period = 50):
    
    return df[column_name].rolling(window=period).mean()

### Calculate EMA

In [30]:
def calculate_EMA20(df, column_name: str = 'close', period: int = 20):

    return df[column_name].ewm(span=period, adjust=False).mean()

def calculate_EMA50(df, column_name: str = 'close', period: int = 50):

    return df[column_name].ewm(span=period, adjust=False).mean()

In [33]:
df_BTC['SMA_20'] = calculate_SMA20(df_BTC)
df_BTC['SMA_50'] = calculate_SMA50(df_BTC)

df_BTC['EMA_20'] = calculate_EMA20(df_BTC)
df_BTC['EMA_50'] = calculate_EMA50(df_BTC)

print(df_BTC)

                       timestamp       open       high        low      close  \
0      2020-01-01 00:00:00+00:00    7195.24    7196.25    7175.46    7177.02   
1      2020-01-01 01:00:00+00:00    7176.47    7230.00    7175.71    7216.27   
2      2020-01-01 02:00:00+00:00    7215.52    7244.87    7211.41    7242.85   
3      2020-01-01 03:00:00+00:00    7242.66    7245.00    7220.00    7225.01   
4      2020-01-01 04:00:00+00:00    7225.00    7230.00    7215.03    7217.27   
...                          ...        ...        ...        ...        ...   
51219  2025-11-05 11:00:00+00:00  101401.97  102110.48  101381.68  102070.61   
51220  2025-11-05 12:00:00+00:00  102070.62  102800.71  101916.77  102673.35   
51221  2025-11-05 13:00:00+00:00  102673.34  103253.39  102291.82  102985.91   
51222  2025-11-05 14:00:00+00:00  102985.90  103470.00  102169.06  103202.30   
51223  2025-11-05 15:00:00+00:00  103202.30  103441.10  102829.51  103046.68   

            volume       SMA_20       S

# Find Pivot Points

In [49]:
def find_pivots(df: pd.DataFrame, window: int = 5, min_gap: int = 4) -> pd.Series:
    """
    Detect pivot (fractal) points in OHLC data, then thin so that in any
    `min_gap` consecutive bars there is at most one pivot (of any type).

    Returns:
        pd.Series: 0 = no pivot, 1 = pivot high, 2 = pivot low, 3 = both.
    """
    pivots = [0] * len(df)

    for candle in range(len(df)):
        # skip edges
        if candle - window < 0 or candle + window >= len(df):
            continue

        pivotHigh = True
        pivotLow = True

        # check neighborhood [candle-window, candle+window]
        for i in range(candle - window, candle + window + 1):
            if df.iloc[candle].low > df.iloc[i].low:
                pivotLow = False
            if df.iloc[candle].high < df.iloc[i].high:
                pivotHigh = False

        if pivotHigh and pivotLow:
            pivots[candle] = 3
        elif pivotHigh:
            pivots[candle] = 1
        elif pivotLow:
            pivots[candle] = 2

    # === Spacing enforcement: at most one pivot in any `min_gap` bars ===
    # Treat any non-zero (1,2,3) as a pivot; keep the first, drop others within the next (min_gap-1) bars.
    piv = np.array(pivots, dtype=int)
    last_kept = -10**9  # sufficiently negative

    # indices of any pivot (1,2,3), in chronological order
    pivot_idx = np.where(piv != 0)[0]

    for idx in pivot_idx:
        if idx - last_kept >= min_gap:
            last_kept = idx      # keep this pivot
        else:
            piv[idx] = 0         # too close to previous pivot → drop

    return pd.Series(piv, index=df.index)

In [50]:
df_BTC['isPivot'] = find_pivots(df_BTC, window=10)

# Plotting to Check

In [51]:
import plotly.graph_objects as go

# If your timestamp isn't already datetime (and tz-naive), normalize it:
if not pd.api.types.is_datetime64_any_dtype(df_BTC['timestamp']):
    df_BTC['timestamp'] = pd.to_datetime(df_BTC['timestamp'], errors='coerce')
if pd.api.types.is_datetime64tz_dtype(df_BTC['timestamp'].dtype):
    df_BTC['timestamp'] = df_BTC['timestamp'].dt.tz_convert('UTC').dt.tz_localize(None)

# Create pivot marker series (tiny offset so markers don’t overlap candles)
df_BTC['pivot_high_price'] = np.where(df_BTC['isPivot'] == 1, df_BTC['high'] * 1.001, np.nan)
df_BTC['pivot_low_price']  = np.where(df_BTC['isPivot'] == 2, df_BTC['low']  * 0.999, np.nan)

# Optional: sort and drop bad rows
df_plot = (df_BTC[1500:1600]
           .dropna(subset=['timestamp', 'open', 'high', 'low', 'close'])
           .sort_values('timestamp')
           .reset_index(drop=True))

# Plot
fig = go.Figure()

# Candlesticks
fig.add_trace(go.Candlestick(
    x=df_plot['timestamp'],
    open=df_plot['open'],
    high=df_plot['high'],
    low=df_plot['low'],
    close=df_plot['close'],
    name='BTC'
))

# Pivot highs (▲ red)
fig.add_trace(go.Scatter(
    x=df_plot['timestamp'],
    y=df_plot['pivot_high_price'],
    mode='markers',
    name='Pivot High',
    marker=dict(symbol='triangle-up', color='red', size=8),
    hovertemplate="Pivot High<br>%{x|%Y-%m-%d %H:%M:%S}<br>Price: %{y:.2f}<extra></extra>"
))

# Pivot lows (▼ green)
fig.add_trace(go.Scatter(
    x=df_plot['timestamp'],
    y=df_plot['pivot_low_price'],
    mode='markers',
    name='Pivot Low',
    marker=dict(symbol='triangle-down', color='lime', size=8),
    hovertemplate="Pivot Low<br>%{x|%Y-%m-%d %H:%M:%S}<br>Price: %{y:.2f}<extra></extra>"
))

fig.update_layout(
    title="BTC — Candlestick with Pivot Points (window=10)",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0)
)

fig.show()

/var/folders/p8/by67wr7139zdvwbp6501h9lh0000gn/T/ipykernel_26338/349701346.py:6: DeprecationWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

